In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from tensorflow import keras
import tensorflow as tf

from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import LearningRateScheduler,ReduceLROnPlateau

from sklearn.model_selection import KFold,GroupKFold
from tensorflow.keras import layers

In [ ]:
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
rb = RobustScaler()
sc = StandardScaler()

**DATA FOR TRAINING & TESTING**

In [ ]:
train = pd.read_csv('../input/ventilator-pressure-prediction/train.csv')
test = pd.read_csv('../input/ventilator-pressure-prediction/test.csv')

**FEATURED ENGNEERING**

In [ ]:
#train['area'] = train.groupby('breath_id')['area'].cumsum()
#train['cross']= train['u_in']*train['u_out']
#train['cross2']= train['time_step']*train['u_out']
train['u_in_lag1'] = train.groupby('breath_id')['u_in'].shift(1).fillna(0)
#train['u_in_lag2'] = train.groupby('breath_id')['u_in'].shift(2).fillna(0)
#train['u_out_lag'] = train.groupby('breath_id')['u_out'].shift(1)
#train['u_in_lag_2'] = train.groupby('breath_id')['u_in'].shift(2)
#train['u_out_lag_2'] = train.groupby('breath_id')['u_out'].shift(2)
train['u_in_diff1'] = train['u_in'] - train['u_in_lag1']
#train['u_in_diff2'] = train['u_in'] - train['u_in_lag2']
#train['u_in_max'] = train['u_in'].groupby(train['breath_id']).transform('max')
#train['u_in_diff_2'] = train['u_in'] - train['u_in_lag_2']
train['u_in_cumsum'] = train['u_in'].groupby(train['breath_id']).cumsum()
#train['expand_mean'] = train.groupby('breath_id')['u_in'].expanding(2).mean().reset_index(level=0,drop=True)
#train['expand_max'] = train.groupby('breath_id')['u_in'].expanding(2).max().reset_index(level=0,drop=True)
#train['expand_std'] = train.groupby('breath_id')['u_in'].expanding(2).std().reset_index(level=0,drop=True)
#train['ewm_u_in_mean'] = train.groupby('breath_id')['u_in'].ewm(halflife=10).mean().reset_index(level=0,drop=True)
#train['ewm_u_in_std'] = train.groupby('breath_id')['u_in'].ewm(halflife=10).std().reset_index(level=0,drop=True)
#train['ewm_u_in_corr'] = train.groupby('breath_id')['u_in'].ewm(halflife=10).corr().reset_index(level=0,drop=True)
#train['R*C'] = train['R'] * train['C']
#train['R'] = train['R'].astype(str)
#train['C'] = train['C'].astype(str)
#train = pd.get_dummies(train)

#test['area'] = test['time_step'] * test['u_in']
#test['area'] = test.groupby('breath_id')['area'].cumsum()
#test['cross']= test['u_in']*test['u_out']
#test['cross2']= test['time_step']*test['u_out']
test['u_in_lag1'] = test.groupby('breath_id')['u_in'].shift(1).fillna(0)
#test['u_in_lag2'] = test.groupby('breath_id')['u_in'].shift(2).fillna(0)
#test['u_out_lag'] = test.groupby('breath_id')['u_out'].shift(1)
#test['u_in_lag_2'] = test.groupby('breath_id')['u_in'].shift(2)
#test['u_out_lag_2'] = test.groupby('breath_id')['u_out'].shift(2)
test['u_in_diff1'] = test['u_in'] - test['u_in_lag1']
#test['u_in_diff2'] = test['u_in'] - test['u_in_lag2']
#test['u_in_max'] = test['u_in'].groupby(test['breath_id']).transform('max')
#test['u_in_diff_2'] = test['u_in'] - test['u_in_lag_2']
test['u_in_cumsum'] = test['u_in'].groupby(test['breath_id']).cumsum()
#test['expand_mean'] = test.groupby('breath_id')['u_in'].expanding(2).mean().reset_index(level=0,drop=True)
#test['expand_max'] = test.groupby('breath_id')['u_in'].expanding(2).max().reset_index(level=0,drop=True)
#test['expand_std'] = test.groupby('breath_id')['u_in'].expanding(2).std().reset_index(level=0,drop=True)
#test['ewm_u_in_mean'] = test.groupby('breath_id')['u_in'].ewm(halflife=10).mean().reset_index(level=0,drop=True)
#test['ewm_u_in_std'] = test.groupby('breath_id')['u_in'].ewm(halflife=10).std().reset_index(level=0,drop=True)
#test['ewm_u_in_corr'] = test.groupby('breath_id')['u_in'].ewm(halflife=10).corr().reset_index(level=0,drop=True)
#test['R*C'] = test['R'] * test['C']
#test['R'] = test['R'].astype(str)
#test['C'] = test['C'].astype(str)
#test = pd.get_dummies(test)

#train = train.fillna(0)
#test = test.fillna(0)

In [ ]:
train

In [ ]:
targets = train['pressure'].to_numpy().reshape(-1, 80, 1)
test_id = test['id']
train.drop(columns = ['id','breath_id','pressure','time_step'], inplace = True)
test.drop(columns = ['id','breath_id','time_step'], inplace = True)

In [ ]:
rb.fit(train)

train_new = rb.transform(train)
test_new = rb.transform(test)

train_re = train_new.reshape(-1, 80, train_new.shape[-1])
test_re = test_new.reshape(-1, 80, test_new.shape[-1])

**MODEL PREPARATION**

In [ ]:
def build_model():
    
    x_input = layers.Input(shape=([80, train_re.shape[-1]]))
    
    x1 = layers.Bidirectional(layers.LSTM(units=440, return_sequences=True))(x_input)
    x2 = layers.Bidirectional(layers.LSTM(units=360, dropout=0.2,return_sequences=True, kernel_initializer='random_normal'))(x1)
    x3 = layers.Bidirectional(layers.LSTM(units=240, dropout=0.2, return_sequences=True, kernel_initializer='random_normal'))(x2)
    x4 = layers.Bidirectional(layers.LSTM(units=180, dropout=0.2, return_sequences=True, kernel_initializer='random_normal'))(x3)
    x5 = layers.Bidirectional(layers.LSTM(units=100, return_sequences=True, kernel_initializer='random_normal'))(x4)
    
    z2 = layers.Bidirectional(layers.GRU(units=240, return_sequences=True))(x2)
    
    z31 = layers.Multiply()([x3, z2])
    z31 = layers.BatchNormalization()(z31)
    z3 = layers.Bidirectional(layers.GRU(units=180, return_sequences=True))(z31)
    
    z41 = layers.Multiply()([x4, z3])
    z41 = layers.BatchNormalization()(z41)
    z4 = layers.Bidirectional(layers.GRU(units=100, return_sequences=True))(z41)
    
    z51 = layers.Multiply()([x5, z4])
    z51 = layers.BatchNormalization()(z51)
    z5 = layers.Bidirectional(layers.GRU(units=64, return_sequences=True))(z51)
    
    x = layers.Concatenate(axis=2)([x5, z2, z3, z4, z5])
    
    x = layers.Dense(units=64, activation='relu')(x)
    
    x_output = layers.Dense(units=1)(x)

    model = tf.keras.Model(inputs=x_input, outputs=x_output, name='Base_Model')
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 0.001), loss=tf.keras.losses.MeanAbsoluteError())
    
    return model

In [ ]:
tf.keras.utils.plot_model(build_model())

**TPU PREPARATION**

In [ ]:
EPOCH = 260
BATCH_SIZE = 512

tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
import math
def exp_decay(epoch):
    initial_lrate = 0.001
    k = 0.006
    lrate = initial_lrate * math.exp(-k*epoch)
    print(f"Learning rate : {lrate}")
    return lrate

lr_schedular = tf.keras.callbacks.LearningRateScheduler(exp_decay)

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', verbose=1, factor=0.87, patience=8)

with tpu_strategy.scope():
    kf = KFold(n_splits=7, shuffle=True)

    for fold, (train_idx, test_idx) in enumerate(kf.split(train_re, targets)):
        print('-'*30, '>', f'Fold {fold+1}', '<', '-'*30)
        X_train, X_valid = train_re[train_idx], train_re[test_idx]

        y_train, y_valid = targets[train_idx], targets[test_idx]
        
        model = build_model()

        # A callback
        call_back = tf.keras.callbacks.ModelCheckpoint(f"NewModel{fold+1}.h5", verbose=1, monitor='val_loss',save_best_only=True)

        his = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=EPOCH, batch_size=BATCH_SIZE, callbacks=[call_back,reduce_lr])
        stats = pd.DataFrame(his.history)
        stats.plot()
        plt.show()
        print("\n\n")

**LOADING MODEL**

In [ ]:
unique_pressures = np.unique(targets)

sorted_pressures = np.sort(unique_pressures)

PRESSURE_STEP = (unique_pressures[1] - unique_pressures[0]).item()

PRESSURE_MIN = sorted_pressures[0].item()
PRESSURE_MAX = sorted_pressures[-1].item()

In [ ]:
PRESSURE_STEP,PRESSURE_MIN,PRESSURE_MAX

In [ ]:
models_paths = ["../input/kfolds/OldModel1.h5","../input/kfolds/OldModel2.h5","../input/kfolds/OldModel3.h5",
                "../input/kfolds/OldModel4.h5","../input/kfolds/OldModel5.h5","../input/kfolds/OldModel6.h5",
                "../input/kfolds/OldModel7.h5","../input/kfolds/OldModel8.h5","../input/kfolds/OldModel9.h5",
                '../input/latest-2fold/Model16049.h5','../input/latest-2fold/Model16275.h5','../input/kfolds/Model16277.h5',
                '../input/kfolds/Model16179.h5']

In [ ]:
#models = [tf.keras.models.load_model(model_path) for model_path in models_paths]

**PREDICTION**

In [ ]:
Ensembles = ['../input/latest-ensembles/submission.csv','../input/latest-ensembles/submission1.csv',
            '../input/latest-ensembles/submission2.csv','../input/latest-ensembles/submission3.csv',
            '../input/latest-ensembles/submission4.csv','../input/latest-ensembles/submission5.csv']

In [ ]:
test_preds = []

for predicted in Ensembles:
    submission_file = pd.read_csv(predicted)
    test_preds.append(submission_file['pressure'].values.reshape(-1,1))

In [ ]:
predictions_median = np.concatenate(test_preds, axis=-1)

In [ ]:
predictions_median.shape

In [ ]:
mean_pre = np.mean(predictions_median, axis = -1)

In [ ]:
mean_pre

In [ ]:
median_pre = np.median(predictions_median, axis = -1)

rounding_pre = np.round((median_pre - PRESSURE_MIN)/PRESSURE_STEP ) * PRESSURE_STEP + PRESSURE_MIN

clipped_pre = np.clip(rounding_pre, PRESSURE_MIN, PRESSURE_MAX)

**SUBMISSION**

In [ ]:
clipped_pre.shape

In [ ]:
submission_file = pd.read_csv("../input/ventilator-pressure-prediction/sample_submission.csv")
submission_file['pressure'] = mean_pre.reshape(-1, )
submission_file.to_csv('submission.csv', index=False)

In [ ]:
submission_file